In [ ]:
print(loaded_model.stages[4].getInputCol())  # Expected input column for the Tokenizer
print(loaded_model.stages[4].getOutputCol())  # Output column from the Tokenizer


Label
label


In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
from pyspark.ml.pipeline import PipelineModel

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Load Local Model and Predict") \
    .getOrCreate()

# Local path where the model is saved
model_save_path = "text_classification_model"

# Load the model from local
loaded_model = PipelineModel.load(model_save_path)

# Create a sample dataframe with new data to test
new_data = [
    ("SK Panglima Terbit, Jenderal Penembak Kucing di Sesko TNI Dimutasi",), 
    ("marvel sorini kolaboratornya membingkainya upaya mendamaikan implikasi fisika modern eksotis pencarian kehidupan bumi memahami energi gelap dampaknya alam semesta salah tantangan terbesar kosmologi fisika fundamental parameter mengatur alam semesta kepadatan energi gelap keberadaan",),
    ("Anggota Polisi di Boyolali Jualan Buku untuk Anak-anak, Paling Murah Rp2.000",)
]

columns = ["Content"]

new_df = spark.createDataFrame(new_data, columns)

# Make predictions
predictions = loaded_model.transform(new_df)

# Get the labels from the StringIndexer used during training
labels = loaded_model.stages[-2].labels  # Assuming StringIndexer is the second-last stage

# Map predictions to their original labels
map_label_udf = udf(lambda x: labels[int(x)], StringType())
predictions = predictions.withColumn("predicted_label", map_label_udf(col("prediction")))

# Show predictions with labels
predictions.select("Content", "predicted_label").show(truncate=False)



24/12/07 11:27:44 WARN StringIndexerModel: Input column Label does not exist during transformation. Skip StringIndexerModel for this column.


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Content                                                                                                                                                                                                                                                                                   |predicted_label|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|SK Panglima Terbit, Jenderal Penembak Kucing di Sesko TNI Dimutasi                              

Traceback (most recent call last):
  File "/home/codespace/.local/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 193, in manager
  File "/home/codespace/.local/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 62, in worker
  File "/home/codespace/.local/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 557, in loads
    length = read_int(stream)
             ^^^^^^^^^^^^^^^^
  File "/home/codespace/.local/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 596, in read_int
    raise EOFError
EOFError


In [15]:
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Test Text Classification Model") \
    .getOrCreate()

# Define the path to the saved model
model_load_path = "hdfs://hadoop-namenode:8020/user/root/output/text_classification_model"

# Load the saved model
model = PipelineModel.load(model_load_path)

# Access the StringIndexer stage to retrieve labels
label_indexer = model.stages[-2]  # Assuming StringIndexer is the second-last stage
labels = label_indexer.labels     # This is the mapping of index to label

# Create test data inline
test_data = [
    ("emarang call for paper annual international conference on islamic studies aicis banjir peminat panitia mencatat paper didaftarkan diseleksi peserta aicis penulis berasal negara aicis kali uin walisongo semarang februari kegiatan mengangkat tema redefining the roles of religion in addressing human crisis encountering peace justice and human rights issues bersamaan tujuh sub tema didiskusikan akademisi kajian keislaman negeri dirjen pendidikan islam m ali ramdhani tema aicis diangkat respon krisis kemanusiaan global belahan dunia gaza palestina tema aicis kali menarik mengangkat tema meredefinisi peran agama krisis kemanusiaan global pemikiran paparan hasilhasil penelitian akademisi negeri diharapkan sumbangsih peradaban dunia ramdhani jakarta rabu januari direktur perguruan keagamaan islam.",),
    (" fbi dikutip militer russia today tampang pria berusia disebar luaskan fbi dunia maya dicap salah buronan dicari farahani rekanrekan intelijen iran direkrut menargetkan pejabat mantan pejabat pemerintahan presiden amerika serikat donald trump salah diincar intelijen militer iran mantan menteri negeri as direktur badan intelijen pusat cia mike pompeo nama pompeo masuh daftar pembunuhan intelijen militer iran politisi partai republik amerika serikat salah perancang serangan drone menewaskan soleimani.",),
    ("The item was okay, but nothing exceptional.",),
]

# Define schema for the test DataFrame
test_columns = ["Content"]

# Create DataFrame from test data
test_df = spark.createDataFrame(test_data, test_columns)

# Make predictions
predictions = model.transform(test_df)

# Map numeric predictions to their label representations
def map_prediction_to_label(prediction):
    return labels[int(prediction)]

# Register the mapping function as a UDF
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
map_to_label_udf = udf(map_prediction_to_label, StringType())

# Add a column with label representations
predictions_with_labels = predictions.withColumn(
    "predicted_label", map_to_label_udf(predictions["prediction"])
)

# Select and display results
predictions_with_labels.select("Content", "prediction", "predicted_label").show(10, truncate=False)

# Stop SparkSession
spark.stop()


24/12/07 11:45:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/12/07 11:45:20 WARN FileSystem: Failed to initialize fileystem hdfs://hadoop-namenode:8020/user/root/output/text_classification_model/metadata: java.lang.IllegalArgumentException: java.net.UnknownHostException: hadoop-namenode


IllegalArgumentException: java.net.UnknownHostException: hadoop-namenode